## __Plato Notebook 2: Optimisation with sediment lubrication, cratonic keels and slab suction__

Thomas Schouten, Whitney Behr, Edward Clennett, Thorsten Becker


#### __0. Prepare Python__

First, we load the required packages.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from plato.optimisation import Optimisation
from plato.plate_torques import PlateTorques
from plato.plot import PlotReconstruction

#### __1. Set up the PlateTorques object__

The below cell loads a `PlateTorques` object for the Müller et al. (2016) reconstruction by providing the ages of interest and the model string identifier "Muller2016", which automatically downloads the relevant files from the GPlately DataServer.

In [ ]:
# Set parameters
# Plate reconstruction
reconstruction_name = "Muller2016" 

# Reconstruction ages of interest
ages = [0]

# Set directory to save the results
results_dir = "02-Results"

# Set up PlateTorques object
M2016 = PlateTorques(reconstruction_name = reconstruction_name, ages = ages, files_dir=results_dir)

#### __2. Sampling and torque calculation__

The below cell calls the `sample_all` method of the `PlateTorques` object to sample seafloor ages at subduction zones and grid points.

Then, the cell calls the `calculate_all_torques` method of the `PlateTorques` object to calculate all torques.

#### __3. Adding trench fill sediments__

The below cells add trench fill sediments to 

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the Haversine distance between two sets of points on Earth.

    :param lat1:    Latitude of the first point in degrees.
    :type lat1:     float
    :param lon1:    Longitude of the first point in degrees.
    :type lon1:     float
    :param lat2:    Latitude of the second point in degrees.
    :type lat2:     float
    :param lon2:    Longitude of the second point in degrees.
    :type lon2:     float

    :return:        The Haversine distance between the two points.
    :rtype:         float
    """
    # Convert latitude and longitude from degrees to radians
    lat1 = np.deg2rad(lat1)
    lon1 = np.deg2rad(lon1)
    lat2 = np.deg2rad(lat2)
    lon2 = np.deg2rad(lon2)

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return c

In [ ]:
# Load Heuret et al. (2012) data
trench_sediment_thickness = pd.read_excel("/Users/thomas/Documents/_Plato/Sediment_lubrication_#2/Heuret2012_data.xlsx")

# Add Heuret et al. (2012) data (which is in km!!) to the sediment thickness column in slabs
for age in M2016.ages:
    for case in M2016.cases:
        if "seds" not in case:
            continue
        
        for i, _ in enumerate(M2016.slabs.data[age][case].lat):
            distance_to_segments = haversine(
                M2016.slabs.data[age][case].loc[i, "lat"],
                M2016.slabs.data[age][case].loc[i, "lon"],
                trench_sediment_thickness.Latitude,
                trench_sediment_thickness.Longitude
            )
            M2016.slabs.data[age][case].loc[i, "sediment_thickness"] = trench_sediment_thickness["Sediment thickness"][distance_to_segments.idxmin()] * 1e3

#### __4. Adding cratonic keels__

The below cells add a grid of lithospheric thickness to the `PlateTorques` object to determine the distribution and extent of cratonic keels.